In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

from Libreria.cargar_configbd import cargar_configbd
from Libreria.cargar_mes_a_procesar import cargar_mes_a_procesar

In [2]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [4]:
mes, año = cargar_mes_a_procesar.leer_csv_en_lista('mes_a_procesar.csv')
mes = 'abr'
año = 2024
print(f"{mes}{año}")

abr2024


In [5]:
tabla_anomala = f"caracteristicas_viajes_{mes}{año}"
tabla_destino = f"corregidos_viajes_{mes}{año}"

In [6]:
def corregir_outliers(df):

    se_corrigio = False

    def deteccion_outliers_isolation_forest(df, umbral_velocidad_maxima):

        # Convertir a numpy array
        X = np.array(df['speed']).reshape(-1, 1)

        # Ajustar Isolation Forest
        clf = IsolationForest(contamination=0.1)
        clf.fit(X)

        # Predecir outliers
        y_pred = clf.predict(X)

        # -1 son outliers, 1 son puntos normales
        df['is_outlier'] = y_pred
        
        # Cambiar el valor de 'is_outlier' a 1 si 'speed' es menor que 40
        df.loc[df['speed'] < umbral_velocidad_maxima, 'is_outlier'] = 1

        se_corrigio = -1 in df['is_outlier'].values

        if not se_corrigio:
            return df, se_corrigio
        
        #Q3 sin outliers 
        Q3_sin_outliers = df[(df['is_outlier'] != -1) | (df['speed'] < umbral_velocidad_maxima)]['speed'].quantile(0.75)

        # Filtrar outliers con velocidad mayor a 40 m/s
        condition = (df['is_outlier'] == -1) | (df['speed'] > umbral_velocidad_maxima)

        # Asignar el nuevo valor a los outliers que cumplen con la condición
        df.loc[condition, 'speed'] = Q3_sin_outliers

        return df, se_corrigio

    umbral_velocidad_maxima = 35
    df_corregido, se_corrigio = deteccion_outliers_isolation_forest(df.copy(), umbral_velocidad_maxima,)

    if se_corrigio:
        # Actualizar las aceleraciones con los datos de velocidades corregidos
        df_corregido['aceleration'] = df_corregido['speed'].diff() / df_corregido['recorded_at'].diff().dt.total_seconds()
        # La primera fila no tendrá aceleración porque no hay un punto anterior para comparar, la establecemos a 0
        df_corregido.loc[0, "aceleration"] = df['aceleration'].iloc[0]

    return df_corregido.drop(columns=['is_outlier'])

In [7]:
# Crear tabla para insertar datos corregidos
consulta_sql =f'''DROP TABLE IF EXISTS corregidos_viajes_{mes}{año};
                CREATE TABLE IF NOT EXISTS corregidos_viajes_{mes}{año}
                    (
                        tripid integer,
                        uid character varying(50),
                        latitude double precision,
                        longitude double precision,
                        distance double precision,
                        speed double precision,
                        aceleration double precision,
                        bearing double precision,
                        recorded_at timestamp without time zone
                    );
                '''

cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()

In [8]:
cur.execute(f"SELECT DISTINCT tripid, uid FROM {tabla_anomala}")
# Obtener los resultados de la consulta
rows = cur.fetchall()
id_viajes = rows.copy()

In [9]:
len(id_viajes)

33044

In [10]:
# ---- Corregir datos
print(f"n_viajes: {len(id_viajes)}")
prog_anterior = 1 # En uno aun no procesa nada
id_viajes = id_viajes[prog_anterior - 1:]
prog = prog_anterior - 1
for id_viaje in id_viajes:
    prog += 1
    print(f"Progreso: {prog}", end="\r")

    # Obtener los ids de todos los viajes
    cur.execute(f"SELECT * FROM {tabla_anomala} where tripid='{id_viaje[0]}' and uid='{id_viaje[1]}' order by recorded_at")
    #print(f"SELECT * FROM {tabla_anomala} where tripid='{id_viaje[0]}' and uid='{id_viaje[1]}' order by recorded_at")

    rows = cur.fetchall()
    puntos_viaje = rows.copy()

    df = pd.DataFrame(puntos_viaje, columns=[desc[0] for desc in cur.description])
    # Se eliminan nulos
    df = df.dropna()
    # Se eliminan posibles datos duplicados
    df = df.drop_duplicates(subset=['recorded_at'], keep='first')
    # Reiniciar los índices
    df.reset_index(drop=True, inplace=True)
    # Se corrigen outliers fijandose en la  velocidad
    df = corregir_outliers(df.copy())

    for index, row in df.iterrows():
        # Construye la consulta SQL con parámetros
        consulta_sql = """
        INSERT INTO {} VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """.format(tabla_destino)

        # Ejecuta la consulta SQL con los valores de la fila actual
        cur.execute(consulta_sql, tuple(row))
 
# Confirma los cambios en la base de datos
conn.commit()

n_viajes: 33044
